In [43]:
# STEP 1: Install dependencies
!pip install gradio groq sentence-transformers faiss-cpu PyPDF2 -q


In [44]:
# STEP 2: Imports
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from typing import List
import gradio as gr
from groq import Groq

In [45]:
# STEP 3: Configure Groq API
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [46]:
# STEP 4: PDF Loading, Chunking, Embeddings, FAISS
def pdf_to_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def text_to_chunks(text: str, chunk_size: int = 400, overlap: int = 100) -> List[str]:
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_texts(texts: List[str]) -> np.ndarray:
    return embed_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

def build_faiss_index(chunks: List[str]):
    embeddings = embed_texts(chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index, embeddings


In [47]:
# STEP 5: Retrieval + Groq Query
def search_index(query: str, chunks: List[str], index, top_k: int = 3):
    query_vec = embed_texts([query])
    scores, indices = index.search(query_vec, top_k)
    return [chunks[i] for i in indices[0]]

def query_groq(query: str, context_chunks: List[str]):
    context = "\n\n".join(context_chunks)
    prompt = f"""
You are a super intelligent and a helpful assistant built and engineered by Parker. Use the following context to answer the question.
If the answer is not in the context, reply with "I don't know."

Context:
{context}

Question:
{query}

Answer:
"""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # valid Groq model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=512
    )
    return response.choices[0].message.content


In [49]:
# STEP 6: RAG Chatbot Logic
chunks, index = None, None  # global vars

def ingest_pdf(file):
    global chunks, index
    raw_text = pdf_to_text(file.name)
    chunks = text_to_chunks(raw_text)
    index, _ = build_faiss_index(chunks)
    return "PDF process Loading....."

def chatbot(user_input, history):
    if chunks is None or index is None:
        return history + [[user_input, "⚠️ Please upload and process a PDF first."]]

    retrieved_chunks = search_index(user_input, chunks, index)
    answer = query_groq(user_input, retrieved_chunks)
    return history + [[user_input, answer]]


In [50]:
# STEP 7: Launch Gradio Chatbot UI
with gr.Blocks() as demo:
    gr.Markdown("## 📚 RAG Chatbot with Groq + FAISS")

    with gr.Row():
        pdf_upload = gr.File(label="📂 Upload your PDF", file_types=[".pdf"])
        process_btn = gr.Button("Process PDF")
        status = gr.Textbox(label="Status", interactive=False)

    chatbot_ui = gr.Chatbot()
    user_input = gr.Textbox(label="Ask a question")
    clear_btn = gr.Button("Clear Chat")

    process_btn.click(ingest_pdf, inputs=pdf_upload, outputs=status)
    user_input.submit(chatbot, [user_input, chatbot_ui], chatbot_ui)
    clear_btn.click(lambda: None, None, chatbot_ui, queue=False)

demo.launch()


/tmp/ipython-input-1790768962.py:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32e59c74c8163ae525.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
